In [5]:
import cv2
import numpy as np
import tensorflow as tf

# Load the trained model (adjust path as needed)
model = tf.keras.models.load_model(r"C:\crop and weed\crop_vs_weed_model.h5")

# Function to classify the cropped image
def classify_crop_or_weed(image):
    # Ensure image has 3 channels (RGB)
    if len(image.shape) == 2:
        image = cv2.cvtColor(image, cv2.COLOR_GRAY2RGB)
    elif image.shape[2] == 1:
        image = cv2.cvtColor(image, cv2.COLOR_GRAY2RGB)

    # Resize the image to match the model's expected input size
    img = cv2.resize(image, (224, 224))  # Change based on `model.input_shape`
    img = img / 255.0
    img = np.expand_dims(img, axis=0)
    
    # Reshape if the model expects flattened input
    if len(model.input_shape) == 2 and model.input_shape[1] != None:
        img = img.reshape((1, -1))  # Flatten the image if needed

    # Predict and return the result
    prediction = model.predict(img)[0][0]
    return "Crop" if prediction < 0.5 else "Weed"

# Load the test image
image_path = r"C:\crop and weed\weed.jpg"
image = cv2.imread(image_path)
clone = image.copy()

# Initialize variables for rectangle drawing
rectangle = []
drawing = False

# Mouse callback function to draw a rectangle and classify the cropped image
def draw_rectangle(event, x, y, flags, param):
    global rectangle, drawing

    if event == cv2.EVENT_LBUTTONDOWN:
        rectangle = [(x, y)]
        drawing = True
    
    elif event == cv2.EVENT_MOUSEMOVE and drawing:
        img_copy = clone.copy()
        cv2.rectangle(img_copy, rectangle[0], (x, y), (0, 255, 0), 2)
        cv2.imshow("Select Area", img_copy)

    elif event == cv2.EVENT_LBUTTONUP:
        rectangle.append((x, y))
        drawing = False

        # Crop the selected area
        x1, y1 = rectangle[0]
        x2, y2 = rectangle[1]
        cropped_image = clone[y1:y2, x1:x2]

        # Prevent empty crops
        if cropped_image.size == 0:
            print("Invalid crop area. Please draw a valid rectangle.")
            return

        # Classify the cropped image
        try:
            result = classify_crop_or_weed(cropped_image)
            color = (0, 255, 0) if result == "Crop" else (0, 0, 255)
        except Exception as e:
            result = "Error"
            color = (255, 0, 0)
            print(f"Error in classification: {e}")

        # Draw the final rectangle and display the classification result
        cv2.rectangle(clone, (x1, y1), (x2, y2), color, 2)
        cv2.putText(clone, result, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, color, 2)
        cv2.imshow("Select Area", clone)

# Show the image and set up mouse callback
cv2.imshow("Select Area", image)
cv2.setMouseCallback("Select Area", draw_rectangle)
cv2.waitKey(0)
cv2.destroyAllWindows()


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
Invalid crop area. Please draw a valid rectangle.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
Invalid crop area. Please draw a valid rectangle.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
